In [1]:
from keras import utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import (
    Activation,
    Dense,
    Dropout,
    Flatten, 
    Conv2D,
    MaxPooling2D)
import numpy as np
import pandas as pd
import tensorflow as tf

Using TensorFlow backend.


In [82]:
# Data structure :'Word POS_tag syntactic_chunk_tag named_entity_tag \n'

# train = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset\eng.train', 'at')
# testa = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset\eng.testa')
# testb = open(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset\eng.testb')
  
def get_words(path, pos): # Add a 'r' before the path for raw string, pos between 0 and 3 see data structure
    data = open(path)
    list_words = []
    new_data = data.readlines()[2:] # Skip first 2 lines
    mylist = list(new_data) # Make the data readable
    for i in range(500): # TODO : CHANGES 20 TO len(mylist)
        if splitter(mylist[i], ' ', pos): # it return None if the line est empty
            list_words.append((splitter(mylist[i], ' ', pos)))
#     print(list_words)
    return list_words

def splitter(line, separator, pos): # pos between 0 and 3 for CONLL2003
    if line.isspace(): # Removing blank lines
        return
    new_sentence = []
    for word in line.split(separator):
        new_sentence.append(word)
    return new_sentence[pos]

def get_dictionary(list_of_words): # Return a dict with how many times a word appears
    dicti = {}
    for word in list_of_words:
        if word in dicti:
            dicti[word] = dicti[word] + 1
        else:
            dicti[word] = 1
    return dicti

def get_reduced_dictionary(dicti, limit): # return a dict with key >= limit
    for v, k in list(dicti.items()):
        if k <= limit:
            del dicti[v]
    return dicti

def get_prefixe(reduced_dict, length_prefixe):
    #TODO : remove all ponctuations signs
    list_prefixes = []
    for word in reduced_dict:
        prefixe = word[:length_prefixe]
        list_prefixes.append(prefixe)
    print('ha', list_prefixes)
    return list_prefixes

def get_suffixe(reduced_dict, length_suffixe):
    #TODO : remove all ponctuations signs
    list_suffixes = []
    for word in reduced_dict:
        util_position = length_suffixe * (-1)
        suffixe = word[util_position:]
        list_suffixes.append(suffixe)
    print('ho', list_suffixes)
    return list_suffixes

def get_capital(list_words):
    # This function returns 2 lists : 1/ If the word have a capital and 2/ If the word is the 1st word of the sentence
    list_capitals = []
    list_first_word = []
    flag_new_sentence = True
    for word in list_words:
        letter = word[0]
        if letter.isupper():
            list_capitals.append(1)
            if flag_new_sentence:
                list_first_word.append(1)
                flag_ne
                w_sentence = False
        else:
            list_capitals.append(0)
        if word == '.': # OU '\n' selon si on les enleve ou pas
            flag_new_sentence = True
    return (list_capitals, list_first_word)



def test():
    list_of_words = get_words(r'C:\Users\gabgu\MSA\PFE\Perceptron\master\Dataset\eng.train', 0)
    print(list_of_words)
    dicti = get_dictionary(list_of_words)
    reduced_dict = get_reduced_dictionary(dicti, 5)
    list_prefixe_2 = get_prefixe(reduced_dict, 2)
    list_prefixe_3 = get_prefixe(reduced_dict, 3)
    list_suffixe_2 = get_suffixe(reduced_dict, 2)
    list_suffixe_3 = get_suffixe(reduced_dict, 3)

#TODO REMOVE ALL DOCSTART LINES    
    
test()

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.', 'Peter', 'Blackburn', 'BRUSSELS', '1996-08-22', 'The', 'European', 'Commission', 'said', 'on', 'Thursday', 'it', 'disagreed', 'with', 'German', 'advice', 'to', 'consumers', 'to', 'shun', 'British', 'lamb', 'until', 'scientists', 'determine', 'whether', 'mad', 'cow', 'disease', 'can', 'be', 'transmitted', 'to', 'sheep', '.', 'Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '"', 'We', 'do', "n't", 'support', 'any', 'such', 'recommendation', 'because', 'we', 'do', "n't", 'see', 'any', 'grounds', 'for', 'it', ',', '"', 'the', 'Commission', "'s", 'chief', 'spokesman', 'Nikolaus', 'van', 'der', 'Pas', 'told', 'a', 'news', 'briefing', '.', 'He', 'said', 'further',

In [80]:
mot = '2'
print(mot.isupper())

False


In [67]:
test={
    "date": 1999,
    "bojour": 3, 
    "popo": 15
}
for x in test:
    print(x)
print(test)

date
bojour
popo
{'date': 1999, 'bojour': 3, 'popo': 15}
